In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import torchvision.datasets as dsets

import torch.nn.utils.prune as prune


In [ ]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print("Using PyTorch version:", torch.__version__,' Device:', DEVICE)

BATCH_SIZE = 32


In [3]:
train_dataset = dsets.FashionMNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(), 
                          download=True)

test_dataset = dsets.FashionMNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

In [ ]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1_2 = nn.Linear(28*28, 512)
        self.fc2_3 = nn.Linear(512, 256)
        self.fc3_4 = nn.Linear(256, 256)
        self.fc4_5 = nn.Linear(256, 256)
        self.fc5_6 = nn.Linear(256, 10)

        
    def forward(self,x):
        x = x.view(-1, 28*28)
        x = self.fc1_2(x)
        x = F.relu(x)
        x = self.fc2_3(x)
        x = F.relu(x)
        x = self.fc3_4(x)
        x = F.relu(x)
        x = self.fc4_5(x)
        x = F.relu(x)
        x = self.fc5_6(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

print(model)

In [9]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print("Train epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image),
                len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

In [10]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset)/ BATCH_SIZE)
    test_accuracy = 100. * correct/len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
EPOCHS = 100
test_accuracy_list = []
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %\n".format(
        epoch, test_loss, test_accuracy))
    test_accuracy_list.append(test_accuracy)

In [ ]:
test_accuracy_list

In [ ]:
import matplotlib.pyplot as plt


epochs = list(range(1, EPOCHS+1)) 


plt.figure(figsize=(10, 5))
plt.plot(epochs, test_accuracy_list, marker='o', linestyle='-', color='b', label='Test Accuracy')
plt.plot(epochs, test_accuracy_list)

plt.title('Test Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
# plt.xticks(epochs)



plt.grid(True)


plt.legend()


plt.show()